In [1]:
import pandas as pd
# import numpy as np
# import random
# from tqdm import tqdm

In [2]:
df_data = pd.read_csv('/egr/research-aidd/menghan1/AnchorDrug/data/level5_beta_trt_cp_24h_10uM.csv')
CellLines = df_data.value_counts('cell_iname').index.to_list()
print(len(CellLines))
print(len(list(set(df_data['SMILES'].to_list()))))
# df_data.value_counts('cell_iname')[:10].plot.bar(rot=30)

117
6645


# pretrain data

In [3]:
# cellline_map = pd.read_csv('/egr/research-aidd/menghan1/AnchorDrug/data/CellLineEncode/use_training_cell_line_expression_features_128_encoded_20240111.csv')
# cell_list = cellline_map['Unnamed: 0'].to_list()
# df_res = df_data[df_data['cell_iname'].isin(cell_list)] #17624 rows x 982 columns
HQ_id_file = '/egr/research-aidd/menghan1/AnchorDrug/data/revise_use_LINCS_HQ_data_pretrain_sample_id.csv'
use_HQ_sample_id = pd.read_csv(HQ_id_file)['x']
df_res = df_data[df_data['sig_id'].isin(use_HQ_sample_id)]
ResDrugs = df_res.drop_duplicates(subset=['SMILES'])['SMILES'].to_list()
print(len(ResDrugs))
print(len(list(set(df_res.cell_iname.to_list()))))
# df_res.to_csv(f'/egr/research-aidd/menghan1/AnchorDrug/data/HQPretrainData/resourceCelllines.csv')
# print(cell_list)
# df_res

2820
100


# finetune/test data

In [4]:
tmp = pd.read_csv('/egr/research-aidd/menghan1/AnchorDrug/HQ_LINCS_retrain/GPS_predictable_genes.csv')
genelist = tmp.x.to_list()

cell_list = ['MCF7', 'PC3', 'A549']

for cell in cell_list:
    print(cell)
    HQ_id_file = f'/egr/research-aidd/menghan1/AnchorDrug/data/revise_use_LINCS_HQ_data_target_cellline_{cell}_sample_id.csv'
    use_HQ_sample_id = pd.read_csv(HQ_id_file)['x']
    df_target = df_data[df_data['sig_id'].isin(use_HQ_sample_id)]
    median = df_target[['SMILES']+genelist].groupby(by='SMILES').median()
    df_target = median

    # load cellline_specific_drug_pool
    tmp = pd.read_csv(f'/egr/research-aidd/menghan1/AnchorDrug/data/cellline_specific_drug_pool_{cell}.csv')
    trainDrugs = tmp['0'].to_list()

    tmp = pd.read_csv(f'/egr/research-aidd/menghan1/AnchorDrug/data/{cell}_internal_val2_data.csv')
    valDrugs = tmp.SMILES.to_list()

    df_train = df_target.loc[trainDrugs]
    print(df_train.shape)
    df_val = df_target.loc[valDrugs]
    print(df_val.shape)
    # print(df_val)
    
    for g in genelist:
        df_train[g] = df_train[g].apply(lambda x: (x > 1.5) * 1 + (x >= -1.5) * 1)
        df_val[g] = df_val[g].apply(lambda x: (x > 1.5) * 1 + (x >= -1.5) * 1)
    # df_train.to_csv(f'/egr/research-aidd/menghan1/AnchorDrug/data/HQdata/{cell}_train.csv')
    # df_val.to_csv(f'/egr/research-aidd/menghan1/AnchorDrug/data/HQdata/{cell}_test.csv')

MCF7
(810, 307)
(134, 307)
PC3
(957, 307)
(141, 307)
A549
(642, 307)
(94, 307)
